In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM

In [2]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
class dataset:
    pass
df = pd.read_csv("03-02-2018.csv")
df.head(5)

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773.0,202,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0.0,38,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527.0,385,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141.0,517,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [4]:
df.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  float64
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

In [6]:
df.shape

(1048575, 80)

In [7]:
df.describe()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,...,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,8.423429e+03,8.049848e+00,1.151421e+07,5.887282e+00,6.983180e+00,3.920838e+02,5.271275e+03,2.104312e+02,8.249200e+00,5.461640e+01,...,2.044231e+00,1.745403e+01,1.229579e+05,6.130972e+04,1.905231e+05,8.305446e+04,4.781197e+06,1.064224e+05,4.863777e+06,4.677017e+06
std,1.516176e+04,4.432629e+00,3.012113e+07,8.903317e+01,2.129580e+02,2.159779e+03,3.088768e+05,2.669497e+02,2.094972e+01,5.762840e+01,...,1.392463e+01,5.179526e+00,2.083099e+06,1.261694e+06,2.761456e+06,1.759832e+06,1.741308e+07,1.462605e+06,1.756808e+07,1.733303e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+01,6.000000e+00,5.160000e+02,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.389000e+03,6.000000e+00,1.124200e+04,3.000000e+00,1.000000e+00,6.300000e+01,1.290000e+02,4.600000e+01,0.000000e+00,4.000000e+01,...,1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.080000e+03,6.000000e+00,2.210150e+06,7.000000e+00,5.000000e+00,3.650000e+02,5.820000e+02,3.260000e+02,0.000000e+00,1.086667e+02,...,3.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553400e+04,1.700000e+01,1.200000e+08,4.315900e+04,6.924100e+04,1.100627e+06,1.010000e+08,1.711000e+03,1.460000e+03,1.460000e+03,...,9.262000e+03,4.400000e+01,1.110000e+08,7.490000e+07,1.110000e+08,1.110000e+08,1.200000e+08,7.590000e+07,1.200000e+08,1.200000e+08


In [8]:
df.dtypes

Dst Port           int64
Protocol           int64
Timestamp         object
Flow Duration      int64
Tot Fwd Pkts       int64
                  ...   
Idle Mean        float64
Idle Std         float64
Idle Max         float64
Idle Min         float64
Label             object
Length: 80, dtype: object

In [9]:
df['Label'].unique()

array(['Benign', 'Bot'], dtype=object)

In [10]:

df.isnull()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1048571,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1048572,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1048573,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
pd.Categorical(df['Flow Byts/s'])




[30597.30523, 135231.31670, 41501.08640, 0.00000, 27100.60726, ..., 796.799143, 833.965404, 796.655744, 797.812144, 797.955823]
Length: 1048575
Categories (458104, float64): [0.000000, 0.008334, 0.008362, 0.008385, ..., 357000000.0, 394000000.0, 480000000.0, inf]

In [12]:
pd.Categorical(df['Flow Pkts/s'])

[113.166178, 10676.156580, 92.915547, 15151.515150, 80.287282, ..., 5.524608, 5.785947, 5.494178, 5.502153, 5.537864]
Length: 1048575
Categories (455755, float64): [0.016669, 0.016671, 0.016671, 0.016671, ..., 1500000.0, 2000000.0, 3000000.0, inf]

In [13]:
df['Flow Pkts/s'].unique()

array([1.13166178e+02, 1.06761566e+04, 9.29155469e+01, ...,
       5.49417755e+00, 5.50215272e+00, 5.53786376e+00])

In [14]:
df.drop('Timestamp',axis=1,inplace=True)

In [15]:
df.shape

(1048575, 79)

In [16]:
#HERE I DROP THE COLOUMS CONTAINING ONLY ZEROS


In [17]:
df.drop(['Bwd PSH Flags','Fwd URG Flags','Bwd URG Flags','CWE Flag Count','Fwd Byts/b Avg','Fwd Pkts/b Avg','Fwd Blk Rate Avg','Bwd Byts/b Avg','Bwd Pkts/b Avg','Bwd Blk Rate Avg'],axis=1,inplace=True)

In [18]:
df.shape

(1048575, 69)

In [19]:
df.isnull().sum()

Dst Port         0
Protocol         0
Flow Duration    0
Tot Fwd Pkts     0
Tot Bwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle Min         0
Label            0
Length: 69, dtype: int64

In [20]:
df['Flow Pkts/s']= df['Flow Pkts/s'].fillna(0)  # removing NAN and INF values
df['Flow Byts/s']= df['Flow Byts/s'].fillna(0)
df['Flow Pkts/s']= df['Flow Pkts/s'].replace(np.Infinity,1)
df['Flow Byts/s']= df['Flow Byts/s'].replace(np.Infinity,1)

In [21]:
pd.Categorical(df['Flow Byts/s']) # now no INF and NAN values

[30597.30523, 135231.31670, 41501.08640, 0.00000, 27100.60726, ..., 796.799143, 833.965404, 796.655744, 797.812144, 797.955823]
Length: 1048575
Categories (458104, float64): [0.000000, 0.008334, 0.008362, 0.008385, ..., 353000000.0, 357000000.0, 394000000.0, 480000000.0]

In [22]:
pd.Categorical(df['Flow Pkts/s'])

[113.166178, 10676.156580, 92.915547, 15151.515150, 80.287282, ..., 5.524608, 5.785947, 5.494178, 5.502153, 5.537864]
Length: 1048575
Categories (455755, float64): [0.016669, 0.016671, 0.016671, 0.016671, ..., 1333333.333, 1500000.000, 2000000.000, 3000000.000]

In [23]:
df.shape

(1048575, 69)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
train = dataset.train

AttributeError: type object 'dataset' has no attribute 'train'

In [ ]:
dataset.train = df.groupby('Label').apply(pd.DataFrame.sample, frac=0.8).reset_index(level='Label', drop=True)
dataset.test = df.drop(dataset.train.index)
dataset.label = dataset.train.Label.copy()

In [ ]:
dataset.label.unique()

In [ ]:
d1 = dataset.train.replace('Benign', 0) # here converting the categorical values to numeric
d2 = d1.replace('Bot', 1)
d6_label = d2.Label.copy()
d6_label.unique()
d6_label.value_counts()

In [ ]:

d6_label.unique()

In [ ]:
dataset.test_label = dataset.test.Label.copy()
dataset.test_label.unique()
a1_label = dataset.test.Label.copy()
a1_label.unique()

In [ ]:

a1 = dataset.test.replace('Benign', 0)
a2 = a1.replace('Bot', 1)
a5_label = a2.Label.copy()
a5_label.unique()
a5_label.value_counts()

In [ ]:

category_variables = ["Protocol"]
for cv in category_variables:
    d2[cv] = d2[cv].astype("category")
    a2[cv] = a2[cv].astype("category")
    
    print("Length of Categories for {} are {}".format(cv , len(d2[cv].cat.categories)))
    print("Categories for {} are {} \n".format(cv ,d2[cv].cat.categories))

In [ ]:
dummy_variables_2labels = category_variables
    
class preprocessing:
    train_labels = pd.get_dummies(d2, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)
    test_labels = pd.get_dummies(a2, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)

In [ ]:
d2.head()


In [ ]:
preprocessing.test_labels.head()

In [ ]:
preprocessing.train_labels.to_csv("preprocessed_train_4_new.csv")
preprocessing.test_labels.to_csv("preprocessed_test_4_new.csv")

In [ ]:
preprocessing.train_labels.head()